In [27]:
from bs4 import BeautifulSoup
import pandas as pd

In [28]:
# open the html file with the records and load into the HTML parser
with open('records.html') as f:
    html_text = f.read()
soup = BeautifulSoup(html_text,'html.parser')

### Parse Page

In [29]:
# get the columns
columns = list()
for column_name in soup.find_all('th'):
    columns.append(column_name.text)

In [30]:
class Record():
    def __init__(self, track, cc, time, player, nation, date, duration, character, vehicle,tires, glider, splits, mushroom, coin, hybrid_used):
        self.track = track
        self.cc = cc
        self.time = time
        self.player = player
        self.nation = nation
        self.date = date
        self.duration = duration
        self.character = character
        self.vehicle = vehicle
        self.tires = tires
        self.glider = glider
        self.splits = splits
        self.mushroom = mushroom
        self.coin = coin
        self.hybrid_used = hybrid_used
    def __str__(self):
        return f'{self.track} {self.cc} {self.time} {self.player} {self.nation} {self.date} {self.duration} {self.character} {self.vehicle} {self.tires} {self.glider} {self.splits} {self.mushroom} {self.coin} {self.hybrid_used} '

In [31]:
def parse_splits(splits_list):
    for i, split in enumerate(splits_list):
        splits_list[i] = float(split[2:-1])
    return splits_list
def parse_mushrooms(mushrooms_list):
    for i, mushroom in enumerate(mushrooms_list):
        if i == 0:
            mushrooms_list[i] = int(mushroom[2])
        elif i == len(mushrooms_list)-1:
            mushrooms_list[i] = int(mushroom[0])
        else:
            mushrooms_list[i] = int(mushroom)
    return mushrooms_list
def parse_coins(coins_list):
    for i, coin in enumerate(coins_list):
        if i == 0:
            coins_list[i] = int(coin[1])
        elif i == len(coins_list)-1:
            coins_list[i] = int(coin[0])
        else:
            coins_list[i] = int(coin[0])
    return coins_list
def parse_mouseover(mouseover_text):
    data = mouseover_text.split('(')[1].split(',')[1:]
    # print(data)
    splits_list = data[:-2]
    splits = parse_splits(splits_list)
    mushrooms_list = data[-1].split('-')
    mushrooms = parse_mushrooms(mushrooms_list)
    coins_list = data[-2].strip().split('-')
    parse_coins(coins_list)
    coins = coins_list
    return splits, mushrooms,coins

In [32]:
records = list()
rows = soup.find_all('tr')
tied_times = {
    'Mario Circuit': 3,
    'GCN Baby Park': 3,
    'Big Blue': 3,
    'Tour Sydney Sprint': 3
}
def number_of_times(course_name):
    if course_name in tied_times: return tied_times[course_name]
    else: return 2
i = 1
while i < len(rows)-4:
    course_name = ''
    time_150 = 0
    hybrid_150 = False
    main_row = rows[i]
    cols = main_row.find_all('td',recursive=False)
    # if cols[0].td is not None:
    #     print(cols[0].td)
    for j, col in enumerate(main_row.find_all('td',recursive=False)):
        if j==0:
            if col.td is not None:
                course_name = col.td.text
        elif j==1:
            time_150 = col.text
            if col.img is not None:
                hybrid_150 = True
        elif j ==2:
            player_150 = col.text
        elif j ==3:
            country_150 = col.img.get_attribute_list('title')[0]
        elif j ==4:
            date_150 = col.text
        elif j ==5:
            duration_150 = col.text
        elif j==6:
            character_150 = col.text
        elif j==7:
            kart_150 = col.text
        elif j==8:
            wheels_150 = col.text
        elif j==9:
            glider_150 = col.text
        elif j==10:
            mouseover = col.img.get_attribute_list('onmouseover')[0]
            splits_150, mushrooms_150, coins_150 = parse_mouseover(mouseover)
    records.append(Record(course_name,150,time_150,player_150,country_150,date_150, duration_150, character_150,kart_150,wheels_150,glider_150,splits_150,mushrooms_150,coins_150,hybrid_150))
    if course_name in tied_times:
        cc_200_row = rows[i+4]
    else:
        cc_200_row = rows[i+3]
    for j, col in enumerate(cc_200_row.find_all('td',recursive=False)):
        if j==0:
            time_200 = col.text
            if col.img is not None:
                hybrid_200 = True
        elif j ==1:
            player_200 = col.text
        elif j ==2:
            country_200 = col.img.get_attribute_list('title')[0]
        elif j ==3:
            date_200 = col.text
        elif j ==4:
            duration_200 = col.text
        elif j==5:
            character_200 = col.text
        elif j==6:
            kart_200 = col.text
        elif j==7:
            wheels_200 = col.text
        elif j==8:
            glider_200 = col.text
        elif j==9:
            mouseover = col.img.get_attribute_list('onmouseover')[0]
            splits_200, mushrooms_200, coins_200 = parse_mouseover(mouseover)
    
    
    records.append(Record(course_name,200,time_200,player_200,country_200,date_200, duration_200, character_200,kart_200,wheels_200,glider_200,splits_200,mushrooms_200,coins_200,hybrid_200))
    
    i += number_of_times(course_name)+2
    # print(i)
    
    

In [33]:
# baby park is an edge case for the parsing, check it here
for i,record in enumerate(records):
    if record.track != 'GCN Baby Park': continue
    print(record)

GCN Baby Park 150 1'02"142 JenZua Spain 2022-02-20 262 Waluigi Mr. Scooty Leaf Tires Paper Glider [10.337, 8.57, 8.655, 8.745, 8.801, 8.451, 8.583] [1, 1, 0, 0, 0, 1, 0] [7, 3, 0, 0, 0, 0, 0] False 
GCN Baby Park 200 0'44"794 EternalX Canada 2021-08-04 462 Baby Daisy Mr. Scooty Button Parachute [7.816, 6.28, 6.144, 6.154, 6.13, 6.143, 6.127] [2, 1, 0, 0, 0, 0, 0] [7, 3, 0, 0, 0, 0, 0] True 


In [34]:
# build the columns for the pandas dataframe
columns[1] = 'cc'
columns[2] = 'time'
columns.append('mushrooms')
columns.append('coins')
columns.append('hybrid_controls_used')
columns

['Track',
 'cc',
 'time',
 'Player',
 'Nation',
 'Date',
 'Duration',
 'Character',
 'Vehicle',
 'Tires',
 'Glider',
 'Splits',
 'mushrooms',
 'coins',
 'hybrid_controls_used']

In [35]:
pd.set_option('display.max_rows',None)
df_rows = []
for record in records:
    s = pd.Series({
        'Track': record.track,
        'cc': record.cc,
        'time':record.time,
        'Player':record.player,
        'Nation':record.nation,
        'Date':record.date,
        'Duration':record.duration,
        'Character':record.character,
        'Vehicle':record.vehicle,
        'Tires': record.tires,
        'Glider': record.glider,
        'Splits': record.splits,
        'mushrooms': record.mushroom,
        'coins':record.coin,
        'hybrid_controls_used':record.hybrid_used
    })
    df_rows.append(s)
wr_df = pd.DataFrame(data=df_rows)
wr_df
    


,Track,cc,time,Player,Nation,Date,Duration,Character,Vehicle,Tires,Glider,Splits,mushrooms,coins,hybrid_controls_used
0,Mario Kart Stadium,150,"1'34""712",Byron,Australia,2022-07-06,126,Roy,Wild Wiggler,Azure Roller,Paper Glider,"[33.047, 30.906, 30.759]","[1, 1, 1]","[8, 2, 0]",True
1,Mario Kart Stadium,200,"1'07""363",Army,France,2022-02-13,269,Wario,Wild Wiggler,Azure Roller,Paper Glider,"[23.521, 22.069, 21.773]","[1, 1, 1]","[7, 3, 0]",True
2,Water Park,150,"1'40""184",Alberto,Spain,2022-01-06,307,Waluigi,Mr. Scooty,Azure Roller,Paper Glider,"[34.534, 32.837, 32.813]","[1, 1, 1]","[8, 2, 0]",True
3,Water Park,200,"1'10""979",Alberto,Spain,2021-09-25,410,Wario,Biddybuggy,Azure Roller,Paper Glider,"[24.705, 23.096, 23.178]","[1, 1, 1]","[8, 2, 0]",True
4,Sweet Sweet Canyon,150,"1'48""845",Alberto,Spain,2022-02-05,277,Baby Daisy,Biddybuggy,Azure Roller,Parachute,"[37.585, 35.757, 35.503]","[1, 1, 1]","[7, 3, 0]",False
5,Sweet Sweet Canyon,200,"1'21""330",Technical,USA,2022-05-22,171,Wario,Biddybuggy,Button,Flower Glider,"[28.333, 26.541, 26.456]","[1, 1, 1]","[7, 3, 0]",True
6,Thwomp Ruins,150,"1'48""975",Alberto,Spain,2022-06-19,143,Baby Daisy,Biddybuggy,Azure Roller,Paper Glider,"[38.095, 35.464, 35.416]","[1, 1, 1]","[1, 0, 0]",True
7,Thwomp Ruins,200,"1'16""172",Alberto,Spain,2022-06-13,149,Wario,W 25 Silver Arrow,Azure Roller,Paper Glider,"[26.744, 24.744, 24.684]","[1, 1, 1]","[1, 0, 0]",True
8,Mario Circuit,150,"1'45""331",しらぬい,Japan,2022-07-05,127,Waluigi,Biddybuggy,Leaf Tires,Paper Glider,"[35.216, 34.689, 35.426]","[2, 1, 0]","[1, 0, 0]",True
9,Mario Circuit,200,"1'12""925",ろーる,Japan,2022-02-13,269,Heavy Mii,W 25 Silver Arrow,Leaf Tires,Paper Glider,"[25.374, 23.751, 23.8]","[1, 1, 1]","[1, 0, 0]",True
